## Working with Database Metadata

With engines and SQL execution down, we are ready to begin some `Alchemy`. The central element of both `SQLAlchemy Core` and `ORM` is the `SQL Expression Language` which allows for fluent, composable construction of SQL queries. The foundation for these queries are _Python objects that represent database concepts like tables and columns_. These objects are known collectively as __database metadata__.

The most common foundational objects for database metadata in SQLAlchemy are known as `MetaData, Table, and Column`. The sections below will illustrate how these objects are used in both a Core-oriented style as well as an ORM-oriented style.

#### Setting up MetaData with Table objects

When we work with a relational database, the basic structure that we create and query from is known as a `table`. In SQLAlchemy, the `table` is represented by a Python class similarly named `Table`.

To start using the `SQLAlchemy Expression Language`, we will want to have `Table` objects constructed that represent all of the database tables we are interested in working with. Each `Table` may be __declared__, meaning we _explicitly spell out in source code what the table looks like_, or may be __reflected__, which means we _generate the object based on what’s already present in a particular database_. The two approaches can also be blended in many ways.

Whether we will _declare_ or _reflect_ our tables, we start out with a collection that will be where we place our tables known as the `MetaData` object. This object is essentially a `facade around a Python dictionary` that stores a series of `Table` objects keyed to their string name.

In [7]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey

In [2]:
metadata_obj = MetaData()

Having `a single MetaData object` for an entire application is the most common case, represented as a _module-level variable_ in a single place in an application, often in a __models__ or __dbschema__ type of package. There can be `multiple MetaData collections` as well, however it's typically _most helpful if a series of Table objects that are related to each other belong to a single MetaData collection_.

_Once we have a `MetaData` object, we can declare some `Table` objects_. This tutorial will start with the classic SQLAlchemy tutorial model, that of the table `user`, which would for example _represent the users of a website_, and the table `address`, representing a list of _email addresses associated with rows in the user table_. We normally assign each `Table` object to a variable that will be how we will refer to the table in application code.

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

We can observe that the above `Table` construct looks a lot like a `SQL CREATE TABLE statement`; starting with the _table name, then listing out each column, where each column has a name and a datatype_. The objects we use above are:

* __`Table`__ - represents a database table and assigns itself to a `MetaData` collection.

* __`Column`__ - represents a column in a database table, and assigns itself to a `Table` object. The `Column` usually includes _a string name and a type object_. The collection of Column objects in terms of the parent `Table` are typically accessed via an associative array located at `Table.c`.

In [4]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [5]:
user_table.c.keys()

['id', 'name', 'fullname']

* __`Integer, String`__ - these classes represent `SQL datatypes` and can be passed to a `Column` _with or without necessarily being instantiated_. Above, we want to give a length of `30` to the `name` column, so we instantiated `String(30)`. But for `id` and `fullname` we did not specify these, so we can send the class itself.

<!-- create table model with declaration strategy -->

#### Declaring Simple Constraints

The first Column in the above `user_table` includes the `Column.primary_key` parameter which is a shorthand technique of indicating that this `Column` should be part of the _primary key_ for this table. The _primary key_ itself is normally declared implicitly and is represented by the `PrimaryKeyConstraint` construct, which we can see on the `Table.primary_key` attribute on the `Table` object.

In [6]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

The constraint that is most typically declared explicitly is the `ForeignKeyConstraint` object that corresponds to a database _foreign key constraint_. When we declare tables that are related to each other, SQLAlchemy uses the presence of these _foreign key constraint_ declarations not only so that they are emitted within `CREATE statements` to the database, but also to assist in `constructing SQL expressions`.

A `ForeignKeyConstraint` that involves only a single column on the target table is typically declared using a _column-level shorthand notation_ via the `ForeignKey` object. Below we declare a second table `address` that will have a _foreign key constraint_ referring to the `user` table.

In [8]:
address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

The table above also features a third kind of _constraint_, which in SQL is the _`NOT NULL` constraint_, indicated above using the `Column.nullable` parameter.

When using the `ForeignKey` object within a `Column` definition, _we can omit the datatype for that `Column`_; it is __automatically inferred__ from that of the related column, in the above example the `Integer` datatype of the `user_account.id` column.